In [1]:
# -*- coding: utf-8 -*-

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from six.moves import xrange

import numpy as np
import os

In [2]:
image_shape = [10, 10]


def fake_data_generator(num=4):
    # define make direction func
    def mkdir(name=None):
        if os.path.exists(name):
            pass
        else:
            os.mkdir(name)

    # generate images
    mkdir(name='images')
    for i in xrange(num):
        new_array = np.zeros(shape=(image_shape), dtype=np.float32) * num
        np.save(file='images/%s.npy' % i, arr=new_array)
    # generate text
    mkdir(name='text')
    for i in xrange(num):
        new_text = str(i)
        with open('text/%s.txt' % i, mode='w') as text_buffer:
            text_buffer.write(new_text)


fake_data_generator()

In [3]:
import tensorflow as tf
import numpy as np
import os
import sys

# tensorflow < 1.4.0
# from tensorflow.contrib.data import Dataset, Iterator
# tensorflow from 1.4.0 to an uncertain versoin
# from tensorflow.data import Dataset
# tensorflow ==1.8.0
Dataset = tf.data.Dataset

img_dir = 'images'
text_dir = 'text'


def gen():
    index = 0
    max_size = len(os.listdir(img_dir))
    while True:
        img = np.load(os.path.join(img_dir, '%s.npy' % (index)))
        with open(os.path.join(text_dir, '%s.txt' % (index))) as reader:
            txt = reader.read()
        yield (img, txt)
        index = (index + 1) % max_size


# 一般来说，batch_size越小，模型训练效果越快，而且越好；但是耗时长。
batch_size = 2
# if memory in GPU and CPU is totally enough,
# use "Dataset.from_tensor_slices", dataset = Dataset.from_tensor_slices((x, y))
# else use "Dataset.from_generator"
dataset = Dataset.from_generator(generator=gen,
                                 output_types=(tf.float32, tf.int32),
                                 output_shapes=(tf.TensorShape(image_shape), tf.TensorShape([])))
dataset = dataset.batch(batch_size=batch_size)
dataset = dataset.shuffle(buffer_size=2)
# dataset.repeat的count为None/-1时，可以无限循环
dataset = dataset.repeat(count=None)
iterator = dataset.make_initializable_iterator()
x = iterator.get_next()

with tf.Session() as session:
    session.run(iterator.initializer)
    for _ in xrange(num // batch_size):
        print(session.run(x))


(array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.